In [1]:
import pandas as pd
import numpy as np

import os

def load_hotel_reserve():
  customer_tb = pd.read_csv('./data/customer.csv')
  hotel_tb = pd.read_csv('./data/hotel.csv')
  reserve_tb = pd.read_csv('./data/reserve.csv')
  return customer_tb, hotel_tb, reserve_tb


def load_holiday_mst():
  holiday_tb = pd.read_csv('./data/holiday_mst.csv',
                           index_col=False)
  return holiday_tb


def load_production():
  production_tb = pd.read_csv('./data/production.csv')
  return production_tb


def load_production_missing_num():
  production_tb = pd.read_csv('./data/production_missing_num.csv')
  return production_tb


def load_production_missing_category():
  production_tb = pd.read_csv('./data/production_missing_category.csv')
  return production_tb


def load_monthly_index():
  monthly_index_tb = \
    pd.read_csv('./data/monthly_index.csv')
  return monthly_index_tb


def load_meros_txt():
  with open('./data/txt/meros.txt', 'r') as f:
    meros = f.read()
    f.close()
  return meros


In [2]:
customer_tb, hotel_tb, reserve_tb = load_hotel_reserve()

#### 2. 범주 데이터 전처리 기법
- 범주형 데이터로의 변환

    -> 범주형은 가질 수 있는 값의 종류가 정해진 값을 의미

In [3]:
# 호텔 예약 레코드를 사용하여 성별을 불리언형과 범주형으로 변환
customer_tb[['sex_is_man']] = (customer_tb[['sex']] == 'man').astype('bool')

customer_tb['sex_c'] = \
    pd.Categorical(customer_tb['sex'], categories=['man', 'woman'])

customer_tb['sex_c'].cat.codes

customer_tb['sex_c'].cat.categories

customer_tb

,customer_id,age,sex,home_latitude,home_longitude,sex_is_man,sex_c
0,c_1,41,man,35.092193,136.512347,True,man
1,c_2,38,man,35.325076,139.410551,True,man
2,c_3,49,woman,35.120543,136.511179,False,woman
3,c_4,43,man,43.034868,141.240314,True,man
4,c_5,31,man,35.102661,136.523797,True,man
...,...,...,...,...,...,...,...
995,c_996,44,man,34.465648,135.373787,True,man
996,c_997,35,man,35.345372,139.413754,True,man
997,c_998,32,woman,43.062267,141.272126,False,woman
998,c_999,48,woman,38.172800,140.464198,False,woman


- 더미 변수화

    -> 파이썬 머신러닝에서 사용되는 일부 함수에서 범주형을 지원하지 않는 경우가 있음

    -> 이 경우 범주값을 플래그의 집합값으로 변환해야 하며 이를 더미 변수화라 함

    -> 더미 변수를 줄이면 머신러닝 예측모델에서 학습데이터를 줄일 수 있지만, 성능에 영향을 줄 수 있음

In [4]:
# 고객 테이블의 성별을 더미 변수로 만들어 봄
customer_tb['sex'] = pd.Categorical(customer_tb['sex'])

dummy_vars = pd.get_dummies(customer_tb['sex'], drop_first=False)

dummy_vars

,man,woman
0,True,False
1,True,False
2,False,True
3,True,False
4,True,False
...,...,...
995,True,False
996,True,False
997,False,True
998,False,True


- 범주 그룹 단순화

    -> 데이터 수가 극단적으로 적은 범주값을 다른 범주값과 묶는 방법을 범주값의 집약

In [ ]:
# 60세 이상을 age_rank로 집약
customer_tb['age_rank'] = \
    pd.Categorical(np.floor(customer_tb['age']/10)*10)  #age_rank를 범주형으로 전환

customer_tb['age_rank'] = customer_tb['age_rank'].cat.add_categories(['60세 이상']) #’60 이상’ 범주 추가

customer_tb.loc[customer_tb['age_rank']\
                .isin([60.0, 70.0, 80.0]), 'age_rank'] = '60세 이상' #60이상 데이터를 ‘60 이상’ 범주로 집약

customer_tb['age_rank'] = customer_tb['age_rank'].cat.remove_unused_categories() #미사용 범주 제거

customer_tb

,customer_id,age,sex,home_latitude,home_longitude,sex_is_man,sex_c,age_rank
0,c_1,41,man,35.092193,136.512347,True,man,40.0
1,c_2,38,man,35.325076,139.410551,True,man,30.0
2,c_3,49,woman,35.120543,136.511179,False,woman,40.0
3,c_4,43,man,43.034868,141.240314,True,man,40.0
4,c_5,31,man,35.102661,136.523797,True,man,30.0
...,...,...,...,...,...,...,...,...
995,c_996,44,man,34.465648,135.373787,True,man,40.0
996,c_997,35,man,35.345372,139.413754,True,man,30.0
997,c_998,32,woman,43.062267,141.272126,False,woman,30.0
998,c_999,48,woman,38.172800,140.464198,False,woman,40.0


- KNN(K-nearest neighbor algorithm) 기반 결측 범주 보완

    -> 범주형 데이터에 결손이 발생하면 결손의 형태에 따라 적절한 대체 방안을 선택

    -> 결손값 보완 방법
    - 고정값으로 보완
    - 집계값으로 보완
    - 결손이 발생하지 않은 데이터에 기반한 예측값으로 보완
    - 시간관계로 보완
    - 다중대입법으로 보완
    - 최대가능도로 보완

    -> KNN: 이용중인 변수에 기반한 데이터 사이의 거리로부터 대상 데이터에 가까운 k개의 값에서 값을 예측하는 알고리즘

In [ ]:
# knn 예측을 이용해 length와 thickness를 기준으로 type을 보완
production_missc_tb = load_production_missing_category()

from sklearn.neighbors import KNeighborsClassifier  #KNN 객체 준비

production_missc_tb.replace('None', np.nan, inplace=True)   #None 데이터 → nan으로 변환 

train = production_missc_tb.dropna(subset=['type'], inplace=False)  #비결손데이터 추출

test = production_missc_tb\
    .loc[production_missc_tb.index.difference(train.index), :]  #결손데이터 추출

kn = KNeighborsClassifier(n_neighbors=3)

kn.fit(train[['length', 'thickness']], train['type'])   #KNN 모델 학습

test['type'] = kn.predict(test[['length', 'thickness']])    #예측값 계산후 type 보완

test['type']

8      E
26     E
30     E
36     A
41     E
      ..
971    A
980    E
983    B
992    A
996    A
Name: type, Length: 100, dtype: object